# Advent of Code 2023

https://adventofcode.com/2023

## Day 1

In [1]:
sample = '1abc2
pqr3stu8vwx
a1b2c3d4e5f
treb7uchet'

"1abc2\npqr3stu8vwx\na1b2c3d4e5f\ntreb7uchet"

In [2]:
def coordinates(haystack)
  digits = haystack.scan(/\d/).to_a
  [digits[0], digits[-1]].join.to_i
end

:coordinates

In [3]:
def checksum(data)
  data.split("\n").map { |line| coordinates(line) }.sum
end

:checksum

In [4]:
puzzle_1 = File.read('2023-d01a.txt')
checksum(puzzle_1)

55621

In [5]:
puzzle_2 = File.read('2023-d01a.txt')

sample_2 = 'two1nine
eightwothree
abcone2threexyz
xtwone3four
4nineeightseven2
zoneight234
7pqrstsixteen'

NUMBERS = {
  'one' => '1',
  'two' => '2',
  'three' => '3',
  'four' => '4',
  'five' => '5',
  'six' => '6',
  'seven' => '7',
  'eight' => '8',
  'nine' => '9'
}

NUMBER_MATCHER = ['\d', *NUMBERS.keys].join('|')
SCANNABLE_NUMBER_MATCHER = Regexp.new("(?=(#{NUMBER_MATCHER}))")

def verbal_checksum(data)
  data.split("\n").map do |line|
    first_digit_match, *_other_matches, last_digit_match = line.scan(SCANNABLE_NUMBER_MATCHER).flatten
    first_digit = NUMBERS[first_digit_match] || first_digit_match
    last_digit = NUMBERS[last_digit_match] || last_digit_match || first_digit
    [first_digit, last_digit].join.to_i
  end.sum
end

# verbal_checksum(sample_2)

verbal_checksum(puzzle_2)

53592

In [6]:
# scan starts looking for the next match after the end of the previous match.
REGULAR_MATCHER = /(\D\D\D)/
'abcde8fgh'.scan(REGULAR_MATCHER)
# [["abc"], ["fgh"]]

[["abc"], ["fgh"]]

In [7]:

# lookahead matches don't consume the characters they match, so they can overlap.
# scan moves forward by one character after each match.
OVERLAPPING_MATCHER = /(?=(\D\D\D))/
'abcde8fgh'.scan(OVERLAPPING_MATCHER)
# [["abc"], ["bcd"], ["cde"], ["fgh"]]

[["abc"], ["bcd"], ["cde"], ["fgh"]]

## Day 2

In [8]:
d2_sample = 'Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green'

def parse(data)
  data.split("\n").map do |game|
    minimums = {}
    number, results = game.split(':')
    game_id = number.match(/\d+/)[0].to_i

    results.split(';').map do |round|
      counts = {}
      round.split(',').map do |hand|
        number, color = hand.strip.split(' ')
        counts[color] = number.to_i
      end
      minimums.merge!(counts) { |_, old, new| [old, new].max }
    end
    [game_id, minimums]
  end
end

def solve(data)
  parse(data).map do |id, game|
    next id if game['red'] <= 12 && game['green'] <= 13 && game['blue'] <= 14
    0
  end
end

# solve(d2_sample)

d2_input = File.read('2023-d02.txt')
solve(d2_input).sum

2685

In [9]:
def solve_2(data)
  parse(data).sum(0) do |_id, game|
    game.values.reduce(&:*)
  end
end

# solve_2(d2_sample)
solve_2(d2_input)

83707

## Day 3

My instinct is to use a bunch of memory to process the entire file in one go, but I'm choosing to process this line by line as though I might need to do this on something like an ever-growing log file.

In [10]:
d3_sample = '2023-d03-sample.txt'
d3_input = '2023-d03.txt'

SYMBOL_MATCHER = /[^.\d]/

def find_codes(pl, fl, nl)
  match_offset = 0
  fl.scan(/\d+/).map do |num|
    start, width = fl.index(num, match_offset) - 1, num.length + 2
    match_offset = start + width
    if "#{pl[start, width]}#{fl[start, width]}#{nl[start, width]}".match(SYMBOL_MATCHER)
      next num.to_i
    end
    0
  end
end

def solve_3(filename)
  prev_line, focus_line, next_line = '', '', ''
  good_codes = []

  File.foreach(filename) do |line|
    padded_line = ".#{line.chomp}."
    prev_line, focus_line, next_line = focus_line, next_line, padded_line
    next if focus_line.empty?
    prev_line = '.' * focus_line.length if prev_line.empty?
    good_codes += find_codes(prev_line, focus_line, next_line)
  end

  # final line
  good_codes += find_codes(focus_line, next_line, '.' * focus_line.length)

  good_codes
end

# find_codes('.#.......', '.35..633.', '.......!.')

# solve_3(d3_sample)
solve_3(d3_input).sum

# d3_problem = [
#   '.....*..#................506..143........375......77.....155...........400.518...64....773...718..797........694....972.603.....*...........',
#   '....479.795...............*..........800...........*.$.......264*636.......@..............&..*...*.......499...............*...5.20.........',
#   '515...................512.484...*....*...=......390...427...................................644.804.........*...@......-..532............28.'
# ]

# find_codes(*d3_problem)

539713

In [11]:
d3_sample = '2023-d03-sample.txt'
d3_input = '2023-d03.txt'

PART_MATCHER = /^\D*\d+\D+\d+\D*$/

def find_parts(*lines)
  parts_found = []

  lines.each do |line|
    if line[3].match(/\D/)
      applesauce = line
      applesauce[3] = 'Z'
      x = applesauce.match(/(\d+)Z/)
      parts_found << x[1].to_i if x
      x = applesauce.match(/Z(\d+)/)
      parts_found << x[1].to_i if x
    else
      x = line.match(/^.{0,2}\D(\d+)/)
      parts_found << x[1].to_i if x
    end
  end

  raise "Should only be two parts. #{lines}" if parts_found.length != 2

  parts_found[0] * parts_found[1]
end

def find_gears(pl, fl, nl)
  gears_found = []
  match_offset = 0

  while index = fl.index('*', match_offset)
    m_start, m_width = index - 1, 3
    p_start, p_width = index - 3, 7
    match_offset = index + 1
    if "#{pl[m_start, m_width]}Z#{fl[m_start, m_width]}Z#{nl[m_start, m_width]}".match(PART_MATCHER)
      gears_found << find_parts(pl[p_start, p_width], fl[p_start, p_width], nl[p_start, p_width])
    end
  end

  gears_found
end

def solve_3b(filename)
  prev_line, focus_line, next_line = '', '', ''
  gear_ratios = []

  File.foreach(filename) do |line|
    padded_line = "...#{line.chomp}..."
    prev_line, focus_line, next_line = focus_line, next_line, padded_line
    next if focus_line.empty?
    prev_line = '.' * focus_line.length if prev_line.empty?
    gear_ratios << find_gears(prev_line, focus_line, next_line)
  end

  # final line
  gear_ratios << find_gears(focus_line, next_line, '.' * focus_line.length)

  gear_ratios
end

# solve_3b(d3_sample).flatten.sum
solve_3b(d3_input).flatten.sum
# find_parts('789..*9', '...*...', '....602')

# find_parts('..217..', '...*...', '222.333')

# '...8...'.match(/^.{0,2}\D(\d+)/)


84159075

In [12]:
a = '123.456'
a[3] = 'Z'
a

"123Z456"

## Day 4

In [13]:
d4_sample = 'Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11'

# scratchcards = d4_sample.split("\n").map { |line| line.split(':') }.map { |id, cards| [id, cards.split('|').map(&:strip).map(&:split)] }.map { |id, cards| [id, cards[0] & cards[1]] }
# scratchcards = d4_sample.split("\n")
#   .map { |line| line.split(':') }
#   .map { |id, cards| [id, cards.split('|')
#     .map(&:strip)
#     .map(&:split)] }
#   .map { |_id, cards| (cards[0] & cards[1])
#     .length }
#   .sum(0) { |matches| matches == 0 ? 0 : 2 ** (matches - 1) }

def solve_d4a(data)
  data.split("\n")
  .map { |line| line.split(':') }
  .map { |id, cards| [id, cards.split('|')
    .map(&:strip)
    .map(&:split)] }
  .map { |_id, cards| (cards[0] & cards[1])
    .length }
  .sum(0) { |matches| matches == 0 ? 0 : 2 ** (matches - 1) }
end

# solve_d4a(d4_sample)

d4_puzzle = File.read('2023-d04.txt')

# solve_d4a(d4_puzzle)

def solve_d4b(data)
  winners = data.split("\n")
  .map { |line| line.split(':') }
  .map { |id, cards| [id, cards.split('|')
    .map(&:strip)
    .map(&:split)] }
  .map { |_id, cards| (cards[0] & cards[1])
    .length }

  counts = [1] * winners.length

  winners.each_with_index do |winner, index|
    next if winner == 0
    winner.times do |i|
      counts[index + 1 + i] += counts[index]
    end
  end

  counts
end

# solve_d4b(d4_sample).sum
solve_d4b(d4_puzzle).sum


14814534

In [14]:
xyz = [3, 5, 2]

xyz[0].times do |i|
  puts i
end

0
1
2


3

## Day 5

### Puzzle 1

I modified my solution in-place to solve part 2 because I started by assuming I could handle the ranges. Part 2 is the first time this AoC that I hit performance reasons to modify my solution.

My part 1 solution used the same definition structure of a range and addend, but I didn't need to bother with any of the operations on ranges.

Puzzle 2

In [15]:
d5_sample = 'seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4'

d5_input = File.read('2023-d05.txt')

def parse_d5(data)
  data.split("\n\n")
end

def range_intersection(a, b)
  # puts "r_i: #{a}, #{b}"
  bottom, top = [a.begin, b.begin].max, [a.end, b.end].min
  return nil unless bottom <= top
  Range.new(bottom, top)
end

def range_slide(range, offset)
  Range.new(range.begin + offset, range.end + offset)
end

def range_rejection(a, subtrahend)
  # puts "r_r: #{a}, #{subtrahend}"
  [Range.new(a.begin, [subtrahend.begin - 1, a.end].min), Range.new([subtrahend.end + 1, a.begin].max, a.end)].reject { |r| r.begin > r.end }
end

def translate(input_ranges, definitions)
  # puts "input_range: #{input_ranges}, definitions: #{definitions}"

  new_ranges = definitions.map do |definition|
    input_ranges.map do |input_range|
        new_range = range_intersection(input_range, definition[:coverage])
        # puts "new_range: #{new_range}"
        next nil unless new_range
        range_slide(new_range, definition[:addend])
    end
  end.flatten.compact

  remaining_ranges = input_ranges.map do |input_range|
    definitions.reduce([input_range]) do |ranges, definition|
      ranges.map do |range|
        # puts "range: #{range}, definition: #{definition}"
        # puts "rejected_range: #{range_rejection(range, definition[:coverage])}"
        range_rejection(range, definition[:coverage])
      end.flatten
    end
  end.flatten.compact

  result = (new_ranges + remaining_ranges)

  # puts "new_ranges: #{new_ranges}"
  # puts "remaining_ranges: #{remaining_ranges}"
  # puts "result: #{result}"

  result
end

def solve_d5(data, num=Float::INFINITY)
  # puts "num: #{num}"
  normalized_data = parse_d5(data).map { |chunk| chunk.split(':') }.map { |_label, definition| definition.strip.split("\n").map { |line| line.split.map(&:to_i) } }
  seeds = normalized_data.shift[0].each_slice(2).map { |x, y| Range.new(x, x + y - 1) }
  maps = normalized_data.map { |definitions| definitions.map { |definition| {coverage: Range.new(definition[1], definition[1] + definition.last - 1), addend: definition.first - definition[1] } } }
  seed_locations = seeds.first([num, seeds.length].min).map { |seed| maps.reduce([seed]) do |points, definitions|
    # puts "points: #{points}, definitions: #{definitions}"
    nx = translate(points, definitions)
    # puts "nx: #{nx}"
    nx
  end }
  # puts "seed_locations: #{seed_locations.length}"
  seed_locations
end

# translate([{coverage: [Range.new(9, 10), Range.new(20, 20)], addend: 0}], [{coverage: Range.new(5, 15), addend: 10}, {coverage: Range.new(19, 21), addend: 50}])

# solve_d5(d5_sample).flatten.map { |range| range.begin }.min
solve_d5(d5_input).flatten.map { |range| range.begin }.min

24261545

In [16]:
require 'benchmark'

Benchmark.bm do |x|
  x.report { solve_d5(d5_input, 1) }
  x.report { solve_d5(d5_input, 2) }
  x.report { solve_d5(d5_input, 4) }
  x.report { solve_d5(d5_input, 8) }
  x.report { solve_d5(d5_input, 16) }
  # x.report { solve_d5(d5_input, 32) }
  # x.report { solve_d5(d5_input, 64) }
  # x.report { solve_d5(d5_input, 128) }
  # x.report { solve_d5(d5_input, 256) }
end

       user     system      total        real
   0.001499   0.000059   0.001558 (  0.001517)
   0.005690   0.000152   0.005842 (  0.005951)
   0.008767   0.000122   0.008889 (  0.008940)
   0.015435   0.000296   0.015731 (  0.015814)
   0.020104   0.000526   0.020630 (  0.020961)


[#<Benchmark::Tms:0x000000010cff38b0 @label="", @real=0.0015170000260695815, @cstime=0.0, @cutime=0.0, @stime=5.9000000000003494e-05, @utime=0.001499000000000028, @total=0.0015580000000000316>, #<Benchmark::Tms:0x000000010db8b240 @label="", @real=0.0059510000282898545, @cstime=0.0, @cutime=0.0, @stime=0.00015199999999998548, @utime=0.005689999999999973, @total=0.005841999999999958>, #<Benchmark::Tms:0x000000010db8cc30 @label="", @real=0.00893999997060746, @cstime=0.0, @cutime=0.0, @stime=0.00012200000000001099, @utime=0.00876699999999997, @total=0.00888899999999998>, #<Benchmark::Tms:0x000000010cff27d0 @label="", @real=0.015813999983947724, @cstime=0.0, @cutime=0.0, @stime=0.0002959999999999907, @utime=0.015434999999999977, @total=0.015730999999999967>, #<Benchmark::Tms:0x000000010db8eb20 @label="", @real=0.020961000001989305, @cstime=0.0, @cutime=0.0, @stime=0.0005259999999999987, @utime=0.02010400000000001, @total=0.02063000000000001>]

## Day 6

### Puzzle 1

In [38]:
d6_sample = 'Time:      7  15   30
Distance:  9  40  200'

d6_input = 'Time:        51     92     68     90
Distance:   222   2031   1126   1225'

sample_race_records = d6_sample.split("\n").map { |line| line.split(':') }.map { |label, data| [label, data.split.map(&:to_i)] }.to_h
input_race_records = d6_input.split("\n").map { |line| line.split(':') }.map { |label, data| [label, data.split.map(&:to_i)] }.to_h

def distance(charge, time)
   run = (time - charge) * charge
end

def intercepts(distance, time)
  # distance = (time - charge) * charge
  puts "distance: #{distance}, time: #{time}"
  a = 0.5 * (time - Math.sqrt(time * time - 4 * distance))
  b = 0.5 * (time + Math.sqrt(time * time - 4 * distance))
  [a, b]
end

def solve_d6(times, distances)
  # intercepts = times.zip(distances).map { |time, distance| intercepts(distance, time) }.map { |a, b| Range.new(a.ceil, b.floor).size }
  intercepts = times.zip(distances).map { |time, distance| intercepts(distance, time) }.map { |a, b| Range.new((a + 1).floor, (b - 1).ceil) }
  intercepts.reduce(1) { |result, range| result * range.size }
end

# solve_d6(sample_race_records['Time'], race_records['Distance'])
solve_d6(input_race_records['Time'], input_race_records['Distance'])

# distance(10, 30)

distance: 222, time: 51
distance: 2031, time: 92
distance: 1126, time: 68
distance: 1225, time: 90


500346

### Puzzle 2

In [42]:
real_sample_race_records = d6_sample.split("\n").map { |line| line.split(':') }.map { |label, data| [label, [data.delete(' ').to_i]] }.to_h
real_input_race_records = d6_input.split("\n").map { |line| line.split(':') }.map { |label, data| [label, [data.delete(' ').to_i]] }.to_h
solve_d6(real_input_race_records['Time'], real_input_race_records['Distance'])


distance: 222203111261225, time: 51926890


42515755

## Day 7

### Puzzle 1

In [86]:
d7_sample = '32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483'

d7_input = File.read('2023-d07.txt')

RANKS = 'AKQJT98765432'

def d7_parse(data)
  data.split("\n").map { |line| line.split(' ') }
end

def hand_type(hand)
  hand.chars.group_by { |c| c }.values.map(&:length).sort.reverse.append(0).take(2).join.to_i
end

def d7_compare(a, b)
  type_a, type_b = hand_type(a), hand_type(b)

  return type_a <=> type_b unless type_a == type_b

  a.chars.zip(b.chars).each do |a, b|
    return RANKS.index(b) <=> RANKS.index(a) unless a == b
  end

  0
end

def solve_d7(data)
  d7_parse(data).sort { |a, b| d7_compare(a.first, b.first) }.each_with_index.reduce(0) do |result, (hand, index)|
    result + hand[1].to_i * (index + 1)
  end
end

# solve_d7(d7_sample)
solve_d7(d7_input)

(irb):8: warning: already initialized constant Object::RANKS
(irb):8: warning: previous definition of RANKS was here


253954294

### Puzzle 2

In [111]:
P2_RANKS = 'AKQT98765432J'

def d7b_hand_type(hand)
  siblings = hand.chars.group_by { |c| c }
  jokers = siblings.delete('J')
  siblings = siblings.values.map(&:length).sort.reverse.append(0)
  siblings[0] += jokers.length if jokers
  siblings.append(0).take(2).join.to_i
end

def d7b_compare(a, b)
  type_a, type_b = d7b_hand_type(a), d7b_hand_type(b)

  return type_a <=> type_b unless type_a == type_b

  a.chars.zip(b.chars).each do |a, b|
    return P2_RANKS.index(b) <=> P2_RANKS.index(a) unless a == b
  end

  0
end

def solve_d7b(data)
  d7_parse(data).sort { |a, b| d7b_compare(a.first, b.first) }.each_with_index.reduce(0) do |result, (hand, index)|
    result + hand[1].to_i * (index + 1)
  end
end

# solve_d7b(d7_sample)
solve_d7b(d7_input)

(irb): warning: already initialized constant Object::P2_RANKS


254837398